# textgenrnn

* https://github.com/minimaxir/textgenrnn
* https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb

In [1]:
import sys, os
import pandas as pd

In [2]:
data_folder = '/media/student/8d1913cf-1155-47a5-a7db-b9a51f445d8f/student/data'

In [3]:
os.path.exists(data_folder)

True

In [4]:
os.listdir(data_folder)

['yelp_dataset',
 'business.csv',
 'jigsaw-toxic-comment-classification-challenge.zip',
 'restaurants.csv',
 'restaurant_reviews.csv',
 'review.csv']

In [6]:
reviews = pd.read_csv(data_folder + '/restaurant_reviews.csv', nrows=100000)

In [7]:
reviews.head()

,Unnamed: 0,business_id,stars,text
0,3,ikCg8xy5JIg_NGPx-MSIDA,5.0,Went in for a lunch. Steak sandwich was delici...
1,5,eU_713ec6fTGNO4BegRaww,4.0,I'll be the first to admit that I was not exci...
2,6,3fw2X5bZYeW9xCz_zGhOHg,3.0,Tracy dessert had a big name in Hong Kong and ...
3,7,zvO-PJCpNk4fgAVUnExYAA,1.0,This place has gone down hill. Clearly they h...
4,10,8mIrX_LrOnAqWsB5JrOojQ,4.0,"Like walking back in time, every Saturday morn..."


In [8]:
reviews.shape

(100000, 4)

In [9]:
restaurants = pd.read_csv(data_folder + '/restaurants.csv')

In [10]:
restaurants.head()

,Unnamed: 0,business_id,name,categories
0,1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,"Specialty Food, Restaurants, Dim Sum, Imported..."
1,2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"Sushi Bars, Restaurants, Japanese"
2,11,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,"Restaurants, Breakfast & Brunch, Mexican, Taco..."
3,13,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,"Italian, Restaurants, Pizza, Chicken Wings"
4,17,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"Restaurants, Italian"


In [11]:
review = pd.read_csv(data_folder + '/review.csv', nrows=100000)

In [13]:
from textgenrnn import textgenrnn
textgen = textgenrnn()

Using TensorFlow backend.


In [14]:
textgen.generate(5)

 20%|██        | 1/5 [00:00<00:03,  1.19it/s]

/r/partying & steals on Randram in a week and wiped by a friend for a cool treat.



 40%|████      | 2/5 [00:01<00:02,  1.47it/s]

Reddit in the street and starts with a memory.



 60%|██████    | 3/5 [00:01<00:01,  1.55it/s]

John Camp in a specific country complete and original and this is what you get two years.



 80%|████████  | 4/5 [00:03<00:00,  1.01it/s]

A stranger that you have a good substitute of that things that are an acquiring quit the first time really they have been an accident and I see yourself to be all the new school straight that was a gun in a bank to the best side programming and for the ended urparation of store are the birthday and



100%|██████████| 5/5 [00:04<00:00,  1.15it/s]

Black Cops out of the strange of the stripes. I have some story of the same videos the craft of life.



In [19]:
texts = reviews[reviews.stars == 1.0]['text'].values[:20]

In [20]:
texts

array(["This place has gone down hill.  Clearly they have cut back on staff and food quality\r\n\r\nMany of the reviews were written before the menu changed.  I've been going for years and the food quality has gone down hill.\r\n\r\nThe service is slow & my salad, which was $15, was as bad as it gets.\r\n\r\nIt's just not worth spending the money on this place when there are so many other options.",
       "Walked in around 4 on a Friday afternoon, we sat at a table just off the bar and walked out after 5 min or so. Don't even think they realized we walked in. However everyone at the bar noticed we walked in!!! Service was non existent at best. Not a good way for a new business to start out. Oh well, the location they are at has been about 5 different things over the past several years, so they will just be added to the list. SMDH!!!",
       "I cannot believe how things have changed in 3 years. I picked up duck congee sometime in the winter when my hubby was sick.  I was very disappoi

In [22]:
textgen.train_on_texts(texts, num_epochs=2, gen_epochs=2)

Training on 13,829 character sequences.
Epoch 1/2
108/108 [==============================] - 25s 232ms/step - loss: 1.1690
Epoch 2/2
108/108 [==============================] - 26s 243ms/step - loss: 0.9851
####################
Temperature: 0.2
####################
I was and I was a little the food on the food and the food was a burrito and the food was a broke that they were all the food was a burrito and the food was as well. I wanted all of the burrito and she was a good terrible the same thing there are to the beach since for a longer and the food was and

I wanted all of this place of the sandwich was a burrito and the food was good because the sandwich was the food and the same time was a terrible sandwiches the food and the food was a bar for the food was a beautiful the food was a burrito and I wanted all of the food was a terrible for a bar was 

I was told to get our the sandwich for the server was a bad for a burrito. I wanted a good their said $10 for a burrito. I was told t